<a href="https://colab.research.google.com/github/daisuke-shimizu/python-Gakushuin-programming1/blob/main/Programming1_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第13講 モンテカルロシミュレーションによる三目並べ

第12講では、三目並べの盤面を、3行3列のNdarrayで表現する方法を紹介しました。
例えば、

$$
\begin{matrix}
\bigcirc & & \times\\
 & \bigcirc & \bigcirc\\
\times & \times & \times\\
\end{matrix}
$$

のような盤面は、

$$
\begin{bmatrix}
1 & 0 & -1\\
0 & 1 & 1\\
-1 & -1 & -1\\
\end{bmatrix}
$$

と3行3列の行列で表され、この行列は次のNdarrayで表すことができます。

```python
import numpy as np
board = np.array([[1,0,-1], [0,1,1], [-1,-1,-1]])
```
また、三目並べの盤面が表現するNdarrayを引数として、
勝敗を判定する`judge`関数を以下のように定義しました。

```python
def judge(b):
    for r in range(3):
        p = np.sum(b[r, :])
        if p == 3:
            return 1 # 後手勝ち
        if p == -3:
            return -1 # 先手勝ち
    for r in range(3):
        p = np.sum(b[:, r])
        if p == 3:
            return 1 # 後手勝ち
        if p == -3:
            return -1 # 先手勝ち
    p = np.sum(np.diag(b))
    if p == 3:
        return 1
    if p == -3:
        return -1
    p = np.sum(np.diag(np.fliplr(b)))
    if p == 3:
        return 1
    if p == -3:
        return -1
    return 0
```
`judge`関数の返り値は、引数であるNdarrayが表す盤面によって、以下のようになります。
- $\bigcirc$（行列中では$1$）の勝ちであれば、$1$
- $\times$（行列中では$-1$）の勝ちであれば、$-1$
- 勝敗がついていなければ、$0$

上記の盤面の例では、$\times$の勝ちですので、`judge(board)`は$-1$を返します。

また、第5講では、モンテカルロシミュレーションについて学びました。
モンテカルロシミュレーションは、ランダムな試行を多数回実行し、
その結果から予測を行うタイプのシミュレーションです。
第5講では、正方形内に膨大な数の点をランダムに発生させ、円周率の推定を行いました。

モンテカルロシミュレーションは、人工知能の実現においても重要な技術の一つであり、
囲碁や将棋の人工知能ソフトは、モンテカルロシミュレーションを利用しています。

この講では、人間と対戦する三目並べのプログラムを、モンテカルロシミュレーションを使って実装します。

この授業では、三目並べのプログラムの作成に必要となるPythonの知識について述べ、
三目並べプログラムのプロトタイプを作成するところまでをガイドします。
皆さんは、手元にダウンロードしたこのノートブック（.ipynbファイル）上で、
プロトタイプの実装までを完了してください。
実装したプロトタイプは、課題として、提出して下さい。

更に余力があれば、プロトタイプの改良を行い、改良したプログラムと、改良点の説明を、
ノートブックに記載して下さい。
改良は加点の対象とします。



## Pythonについてのいくつかの補足
### タプル
タプルは、要素の並びという点で、リストと非常によく似ています。
表示上は、リストが、要素を四角括弧`[ ]`で囲むのに対し、
タプルでは、丸括弧`( )`で囲みます。
```python
a = [0, 1, 2]
b = (0, 1, 2)
print("a[0] = ", a[0])
print("a[1] = ", a[1])
print("a[2] = ", a[2])
print("b[0] = ", b[0])
print("b[1] = ", b[1])
print("b[2] = ", b[2])
```

In [6]:
a = [0, 1, 2]
b = (0, 1, 2) # タプル
print("a[0] = ", a[0])
print("a[1] = ", a[1])
print("a[2] = ", a[2])
print("b[0] = ", b[0])
print("b[1] = ", b[1])
print("b[2] = ", b[2])

# そのオブジェクトが変更可能かそうではないか

a[0] =  0
a[1] =  1
a[2] =  2
b[0] =  0
b[1] =  1
b[2] =  2


リストとタプルの最も重要な違いは、リストは値も長さも変更可能であるのに対し、
タプルは生成時から一切の変更ができないという点です。
例えば、リスト`a`に対する以下のプログラムは、いずれの行も正しく動作します。
```python
a.append(4)
print(a)
a[3] = 3
print(a)
```

In [3]:
a = [0, 1, 2]
a.append(4)
print(a)
a[3] = 3
print(a)

[0, 1, 2, 4]
[0, 1, 2, 3]


一方、タプル`b`に関する次のプログラムは、どれもエラーになります。
```python
b.append(4)
```
```python
b[2] = 3
```

In [5]:
b.append(4)
b[2] = 3

AttributeError: ignored

### 文字列リストの接合
文字列のリストが与えられた時、任意に指定する区切り文字列を使って、
リストの要素を接合する便利な方法があります。
論より証拠で、次のプログラムを実行してみて下さい。
```python
a = ["This", "is", "a", "pen."]
print(" ".join(a))
```

In [7]:
a = ["This", "is", "a", "pen."]
print(" ".join(a))

This is a pen.


区切り文字列は、スペース`" "`ですが、これは任意に指定することができます。
例えば、
```python
a = ["This", "is", "a", "pen."]
print("_".join(a))
```
とすると、区切りはアンダースコア`_`になります。

In [69]:
a = ["This", "is", "a", "pen."]
print("_".join(a))

This_is_a_pen.


`join()`と対で覚えておいた方が良い便利な関数として、`split()`があります。
`split()`は`join()`の逆で、長い文字列が与えられた時に、
文字列を指定された区切り文字列で分割し、分割の結果をリストとして返します。
```python
b = "This is a pen."
print(b.split(' '))
```

In [70]:
b = "This is a pen."
print(b.split(' '))

['This', 'is', 'a', 'pen.']


### オブジェクトのコピー
次のプログラムの実行結果は、何になると思いますか？
`[0, 1, 2]`でしょうか？
`[0, 1, 2, 3]`でしょうか？
```python
a = [0, 1, 2]
b = a
a.append(3)
print(b)
```

In [71]:
a = [0, 1, 2]
b = a
a.append(3)
print(b)

[0, 1, 2, 3]


以前の資料でも説明しましたが、Pythonにおいて、
**`名前=オブジェクト`はオブジェクトに名前をつける**という意味です。
```python
a = [0, 1, 2]
```
は、`[0, 1, 2]`という内容を持つリストオブジェクトに、`a`という名前をつけています。
名前をつけたので、これから先は、このオブジェクトを`a`という名前で参照することができます。
更に、
```python
b = a
```
は、名前`a`が指し示すオブジェクト（`[0, 1, 2]`を内容に持ちます）に、
追加で、`b`という名前をつけるという意味です。
したがって、二つの名前`a`と`b`は、いずれも、同じオブジェクトを指しています。
ですので、
```python
a.append(3)
```
は、名前`a`が指し示しているオブジェクトに、数`3`を追加しますが、
`b`は同じオブジェクトの別名ですので、
```python
print(b)
```
を実行すると、`[0, 1, 2, 3]`が表示されるわけです。

`b = a`で本当にしたいことが、
**`a`と同じ内容を持つオブジェクトをコピーして、
コピーによって新しく生成されたオブジェクトに`b`という名前をつける**
ということでしたら、
明示的に、`a`が指し示すオブジェクトのコピーを作らなければなりません。
```python
b = a.copy()
```
とすれば、`a.copy()`により、
`a`が指し示すオブジェクトと同じ内容（`[0, 1, 2]`）を持つオブジェクトのコピーに対して、
名前`b`をつけることができます。
`b = a`を`b = a.copy()`に置き換えて、同じプログラムを試してみて下さい。

In [72]:
a = [0, 1, 2]
b = a.copy()
a.append(3)
print(b)

[0, 1, 2]


この名前付のルールは、Pythonがオブジェクト指向プログラミング言語であることに起因するのですが、
オブジェクト指向ではないCなど、普及したプログラミング言語との互換性と、
プログラミングの利便性を考えて、
数や文字列などのプリミティブなオブジェクトでは、
`copy()`を明示的に使用しなくても、コピーが作成されます。

次のプログラムの実行結果は、何になると思いますか？
`0`でしょうか？
`1`でしょうか？

```python
a = 0
b = a
a += 1
print(a)
print(b)
```

In [73]:
a = 0
b = a
a += 1
print(a)
print(b)

1
0


次のプログラムも試してみて下さい。
```python
a = "abc"
b = a
a += 'd'
print(a)
print(b)
```

In [74]:
a = "abc"
b = a
a += 'd'
print(a)
print(b)

abcd
abc


### 最大値・最小値
リストの要素の最大値・最小値は、`max()`と`min()`の二つの関数で求めることができます。
```python
a = [3, 1, 4, 1, 5, 9, 2]
print(max(a))
print(min(a))
```

In [8]:
a = [3, 1, 4, 1, 5, 9, 2]
print(max(a))
print(min(a))

9
1


では、`a`の要素$x$で、関数値$f(x)$を最大・最小にする$x$を求めましょう。
```python
def f(x):
    return (x-5)**2

print(max(a, key=f))
print(min(a, key=f))
```

In [9]:
def f(x):
    return (x-5)**2
 
print(max(a, key=f))
print(min(a, key=f))

1
5


別の例を見てみましょう。
リスト`b`の要素はリストで、平均が最大・最小となる要素（リスト）を求めます。
```python
import numpy as np
b = [[3,1,4], [1,5,9], [2,6,5,3], [5,8]]
print([np.average(x) for x in b])
print(max(b, key=np.average))
print(min(b, key=np.average))
```

In [10]:
import numpy as np
b = [[3,1,4], [1,5,9], [2,6,5,3], [5,8]]
print([np.average(x) for x in b])
print(max(b, key=np.average))
print(min(b, key=np.average))

[2.6666666666666665, 5.0, 4.0, 6.5]
[5, 8]
[3, 1, 4]


辞書オブジェクト
```python
c = {0:3, 1:1, 2:4, 3:1, 4:5, 5:9, 6:2}
```
に対して、キーから値を得るには、例えば、`c[0]`とすれば良いのですが、
関数`c.get()`を用いることもできます。
つまり、`c[0]`でも、`c.get(0)`でも、同じ結果`3`を得ることができます。

`c.get`関数を使えば、値がそれぞれ最大・最小となるキーは、次のプログラムで求めることができます。
```python
print(max(c, key=c.get))
print(min(c, key=c.get))
```

In [11]:
c = {0:3, 1:1, 2:4, 3:1, 4:5, 5:9, 6:2}

print(max(c, key=c.get))
print(min(c, key=c.get))

5
1


## 三目並べプログラムの作成
いよいよ、人間と対戦する三目並べのプログラムを作成していきます。
以下では、プログラムの完成に至るまでのステップをガイドします。

### 盤面Ndarrayの表示
盤面を表現するNdarrayの表示から盤面を理解することはできますが、
見やすいとは言えないので、
Ndarrayを引数として、盤面を見やすく表示する関数`show`を作成します。

```python
board = np.array([[1,0,-1], [0,1,1], [-1,-1,-1]])
show(board)
```
によって、以下を表示させることが目的です。

$$
\begin{matrix}
& 0 & 1 & 2\\
0 & \rm O & & \rm X\\
1 & & \rm O & \rm O\\
2 & \rm X & \rm X & \rm X\\
\end{matrix}
$$

In [13]:
def convert(x):
  if x == 1:
    return 'O'
  elif x == -1:
    return 'X'
  else:
    return ' '

def show(b):
  print("  0 1 2")
  for i in range(3):
    print(i, ' '.join([convert(x) for x in b[i]]))


board = np.array([[1,0,-1], [0,1,1], [-1,-1,-1]])
print(board)
show(board)

[[ 1  0 -1]
 [ 0  1  1]
 [-1 -1 -1]]
  0 1 2
0 O   X
1   O O
2 X X X


### 着手可能なマス目の列挙
モンテカルロシミュレーションでは、
まだ着手がされていない（$\bigcirc$・$\times$のいずれもが入力されていない）マス目をランダムに選んで、
勝負がつくか、全てのマス目が埋まるまで、交互に$\bigcirc$と$\times$を埋めていきます。
そのため、現在どのマス目に新たに印をつけられるか、知る手段が必要になります。

盤面を表すNdarrayを引数として、成分が0の位置を列挙する関数`blank`を作成します。
成分0の要素の位置は、行番号と列番号のタプルで表現します。
したがって、`blank`の返り値はタプルのリストです。

```python
board = np.array([[1,0,-1], [0,1,1], [-1,-1,-1]])
blank(board)
```
を実行すると、次のリストを得ます。
```python
[(0, 1), (1, 0)]
```

In [17]:
def blank(b):
  s = []
  for i in range(3):
    for j in range(3):
      if b[i, j] == 0:
        s.append((i, j))
  return s
print(blank(board))

[(0, 1), (1, 0)]


### ランダムな着手
次の手順をプログラミングして下さい。

1. 現在の盤面がNdarray`b`で表されているものとして、
`s = blank(b)`からランダムにマス目を選びます。
1. 選んだマス目に着手して、`b`と`s`を更新します。
1. `judge(b)`により、勝敗を判定します。

In [18]:
def judge(b):
    for r in range(3):
        p = np.sum(b[r, :])
        if p == 3:
            return 1 # 後手勝ち
        if p == -3:
            return -1 # 先手勝ち
    for r in range(3):
        p = np.sum(b[:, r])
        if p == 3:
            return 1 # 後手勝ち
        if p == -3:
            return -1 # 先手勝ち
    p = np.sum(np.diag(b))
    if p == 3:
        return 1
    if p == -3:
        return -1
    p = np.sum(np.diag(np.fliplr(b)))
    if p == 3:
        return 1
    if p == -3:
        return -1
    return 0




In [45]:
import random 
b = np.array([[1,0,-1], [0,1,1], [0,-1,-1]])
show(b)
s = blank(b)
i, j =random.choice(s)
b[i, j] = 1
print(s)
s.remove((i, j))
judge(b)

  0 1 2
0 O   X
1   O O
2   X X
[(0, 1), (1, 0), (2, 0)]


0

### プレイアウト
勝敗がつくか、着手可能なマス目がなくなる（従って、引き分けとなる）まで、ランダムな着手を続けることを、
**プレイアウト**と呼びます。
プレイアウトを行う関数`playout`を作成して下さい。
- 簡単のために、コンピュータは$\bigcirc$を持つと仮定します。
- 関数定義の外に盤面を表現するNdarray`board`を用意し、
`playout()`の定義中では`board`のコピーを作成して、利用します。
- `playout()`の返り値は、着手のリストと、最後の勝敗結果とします。

In [50]:
def playout():
  b = board.copy()
  s = blank(b)
  p = judge(b)
  h = [] # 着手のリスト
  t = 1 #手番 1 or -1
  while p == 0 and s != []:
    i, j = random.choice(s)
    h.append((i, j))
    b[i, j] = t
    t *= -1 # -1をかけて手番を変える
    s.remove((i, j))
    p = judge(b)
  return(h, p)
board = np.array([[1,0,-1], [0,1,1], [0,-1,-1]])
show(board)
h, p = playout()
print(h)
print(p)

  0 1 2
0 O   X
1   O O
2   X X
[(2, 0), (1, 0), (0, 1)]
0


### 次の一手の決定
プレイアウトを$N$回（例えば、$N = 1000$）繰り返し、
勝ち数と負け数の差（`balnace`）が最も大きい第一手を選ぶ関数`next()`を作成して下さい。
- 関数内に、辞書型オブジェクト`balance`を用意します。
`balance`は、第一手をキー、$(勝ち数 - 負け数)$を値として、対応づけます。

In [56]:
N = 1000
def next():
  balance = {}
  for _ in range(N):
    h, p = playout()
    if h[0] in balance:
      balance[h[0]] += p
    else:
      balance[h[0]] = p
  print(balance)
  return max(balance, key=balance.get)

In [60]:
board = np.array([[1,0,-1], [0,1,1], [0,-1,-1]])
show(board)
print(next())

  0 1 2
0 O   X
1   O O
2   X X
{(1, 0): 311, (0, 1): -182, (2, 0): 189}
(1, 0)


### 人間 VS プログラムの対戦
次の3つのセルを作成し、これらのセルを適切な順番で、交互に実行することにより、
人間とプログラムが対戦できるようプログラミングして下さい。
- 盤面を初期化するセル
- 人間が着手するセル
- `next`関数を利用して、プログラムが着手するセル



In [94]:
# 盤面の初期化
board = np.zeros([3,3])
show(board)

  0 1 2
0      
1      
2      


In [82]:
# 人間の着手
board[2, 1] = -1
show(board)

  0 1 2
0 X O X
1 X O  
2 O X  


In [83]:
# プログラムの着手
i, j = next()
board[i, j] = 1
show(board)

{(1, 2): 0, (2, 2): 0}
  0 1 2
0 X O X
1 X O O
2 O X  


# 追加機能
### 自分が打つと、プログラムが自動的に次の手を打ってくれる、game関数を作成しました。
#### 遊び方
- 代入可能な行と列番号を一回ずつ入力します。

### 未完成部分
- 代入不可能な数値が入力された時の条件分岐
- 自分が勝った場合の条件分岐（仕様上、勝つことはほとんど不可能であるが）

In [129]:
def game():
  s = blank(board)
  p = judge(board)
  h = [] # 着手のリスト
  t = 1
  while p == 0 or s != []:
    s = blank(board)
    p = judge(board)
    if t == 1:
      a = input()
      b = input()
      board[int(a), int(b)] = -1
      show(board)
      h.append((int(a), int(b)))
      s.remove((int(a), int(b)))
      t*= -1
      s = blank(board)
      p = judge(board)
      if s == []:
        print('引き分けです')
        break
    i, j = next()
    board[i, j] = 1
    show(board)
    h.append((i, j))
    s.remove((i, j))
    t*= -1
    s = blank(board)
    p = judge(board)
    if p ==1:
      print('コンピュータの勝ち')
      break
    if s == []:
      print('引き分けです')
      break
    print('---')
board = np.zeros([3,3])
game()



0
0
  0 1 2
0 X    
1      
2      
{(1, 1): -20, (1, 2): -39, (2, 2): -39, (0, 1): -68, (1, 0): -67, (2, 0): -43, (0, 2): -47, (2, 1): -57}
  0 1 2
0 X    
1   O  
2      
---
0
1
  0 1 2
0 X X  
1   O  
2      
{(2, 2): -73, (1, 0): -21, (0, 2): 47, (2, 1): -111, (1, 2): -40, (2, 0): -15}
  0 1 2
0 X X O
1   O  
2      
---
2
0
  0 1 2
0 X X O
1   O  
2 X    
{(1, 0): 79, (2, 2): -98, (1, 2): 1, (2, 1): -150}
  0 1 2
0 X X O
1 O O  
2 X    
---
1
2
  0 1 2
0 X X O
1 O O X
2 X    
{(2, 2): 0, (2, 1): 0}
  0 1 2
0 X X O
1 O O X
2 X   O
---
2
1
  0 1 2
0 X X O
1 O O X
2 X X O
引き分けです


In [90]:
a = input()
b = input()
print(int(a)*int(b))

8
9
72
